In [31]:
import os
os.chdir("x:\DL\Projects\MLOPs")
%pwd

'x:\\DL\\Projects\\MLOPs'

In [32]:
import yaml
import os
import shutil
from sklearn.model_selection import train_test_split

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data: Path
    unzip_dir: Path

@dataclass(frozen=True)
class DataSetConfig:
    base_Data_path: Path
    classes: list
    new_data_path: Path
    dataset_name: str
    

In [34]:
from scripts.MLOPs.constants import *
from scripts.MLOPs.utils.common import read_yaml, create_directories
#from scripts.MLOPs.entity.config_entity import *

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_dataingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data= config.local_data,
            unzip_dir= config.unzip_dir
            )
        return data_ingestion_config
    
    def get_Dataset_config(self)-> DataSetConfig:
        config = self.config.custom_dataset
        dataset_config = DataSetConfig(
            base_Data_path= config.base_Data_path,
            classes= config.classes,
            new_data_path= config.new_data_path,
            dataset_name=config.dataset_name
        )
        return dataset_config

In [35]:
import yaml
import os
import shutil
from sklearn.model_selection import train_test_split
from scripts.MLOPs import logger
from scripts.MLOPs.utils.common import read_yaml

class DataSetPreparer:
    def __init__(self, yamlconfig: DataSetConfig):
        self.yamlconfig = yamlconfig

    def indices(self, base_set: list, new_set: list):
        base_set = [item.lower() for item in base_set]
        new_set = [item.lower() for item in new_set]
        new_indices =[]
        for item in new_set:
            indices = [i for i, x in enumerate(base_set) if x == item]
            new_indices.extend(indices)
        logger.info(f"index changes successfully into {new_indices}")
        return new_indices
        

    def filter_files(self, directory, classid_list):
        file_list = []
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                file_path = os.path.join(directory,filename)
                with open(file_path, 'r') as file:
                    content = file.readline().strip().split()
                    if content:
                        class_id = int(content[0])
                        if class_id in classid_list:
                            file_list.append(os.path.splitext(filename)[0])
        logger.info(f"file list has been updated")
        return file_list
    
    def copy_files(self, file_list, source, destination):
        for root, _, files in os.walk(source):
            for file in files:
                filebase, extension = os.path.splitext(file)
                if filebase in file_list:
                    source = os.path.join(root, file)
                    target = os.path.join(destination, file)

                    if not os.path.exists(os.path.dirname(target)):
                        os.makedirs(os.path.dirname(target))

                    shutil.copy2(source, target)
                    logger.info(f"copied {source} to {target}")
    
    def move_files(self, file_list, source, destination):
        for root, _, files in os.walk(source):
            for file in files:
                filebase, extension = os.path.splitext(file)
                if filebase in file_list:
                    source = os.path.join(root, file)
                    target = os.path.join(destination, file)

                    if not os.path.exists(os.path.dirname(target)):
                        os.makedirs(os.path.dirname(target))

                    shutil.move(source, target)
                    logger.info(f"copied {source} to {target}")

    def update_id(self, line, old_to_newid):
        parts = line.strip().split()
        if parts:
            original_id = int(parts[0])
            if original_id in old_to_newid:
                parts[0] = str(old_to_newid[original_id])
        return ' '.join(parts) + '\n'

    def subsetandcopy(self,filelist, source_dir,destination_dir,type,train_ratio:float = 0.7, valid_ratio:float = 0.2):
        ###type can be image or label
        train_files, test_files = train_test_split(filelist, test_size=1-train_ratio, random_state=42)
        valid_ratio_adj = valid_ratio/(1-train_ratio)
        train_files, valid_files = train_test_split(train_files, test_size=valid_ratio_adj, random_state=42)

        self.move_files(train_files,source_dir,os.path.join(destination_dir,"train",type))
        self.move_files(valid_files,source_dir,os.path.join(destination_dir,"valid",type))
        self.move_files(test_files,source_dir,os.path.join(destination_dir,"test",type))
        logger.info(f'files has been subset into train valid and test sets')

    def create_yaml(self, dir_path: str, class_name: list, op_file: str):
        data = {
            'path': dir_path,
            'train': "train/images",
            'val': "valid/images",
            'test': "test/images",
            'nc': len(class_name),
            'names': class_name
        }
        with open (op_file, 'w') as file:
            yaml.dump(data, file, default_flow_style=False)

        
    def create_dataset(self):
        #rread the base yaml
        yaml_path = Path(f"{self.yamlconfig.base_Data_path}/data.yaml")
        print(yaml_path)
        base_yaml = read_yaml(yaml_path)
        new_dir = os.path.join(self.yamlconfig.new_data_path,self.yamlconfig.dataset_name)
        os.makedirs(new_dir, exist_ok=True)
        
        #create folder structure
        for folder in ['train','valid','test']:
            subdir = os.path.join(new_dir, folder)
            os.makedirs(os.path.join(subdir, 'images'), exist_ok=True)
            os.makedirs(os.path.join(subdir, 'labels'), exist_ok=True)
        logger.info("folder structure for the new dataset has been created")

        #set the classes to filter
        desired_classes = self.yamlconfig.classes
        logger.info(f"desired classes are: {desired_classes}")

        base_class = base_yaml.names
        logger.info(f"initial classes are: {base_class}")
        #finding index
        new_indx_wrt_base = self.indices(base_class,desired_classes)
        #filtering train files wrt to index
        filtered_train_filelist = self.filter_files(os.path.join(self.yamlconfig.base_Data_path,"train","labels"),new_indx_wrt_base)
        #filtering valid files wrt to index
        filtered_valid_filelist = self.filter_files(os.path.join(self.yamlconfig.base_Data_path,"valid","labels"),new_indx_wrt_base)
        #filtering test files wrt to index
        filtered_test_filelist = self.filter_files(os.path.join(self.yamlconfig.base_Data_path,"test","labels"),new_indx_wrt_base)
        filtered_filelist = filtered_train_filelist + filtered_valid_filelist + filtered_test_filelist
        #copy the images into train and labels into valid
        self.copy_files(filtered_train_filelist,os.path.join(self.yamlconfig.base_Data_path,"train","images"),os.path.join(new_dir,"train"))
        self.copy_files(filtered_train_filelist,os.path.join(self.yamlconfig.base_Data_path,"train","labels"),os.path.join(new_dir,"valid"))
        self.copy_files(filtered_valid_filelist,os.path.join(self.yamlconfig.base_Data_path,"valid","images"),os.path.join(new_dir,"train"))
        self.copy_files(filtered_valid_filelist,os.path.join(self.yamlconfig.base_Data_path,"valid","labels"),os.path.join(new_dir,"valid"))
        self.copy_files(filtered_test_filelist,os.path.join(self.yamlconfig.base_Data_path,"test","images"),os.path.join(new_dir,"train"))
        self.copy_files(filtered_test_filelist,os.path.join(self.yamlconfig.base_Data_path,"test","labels"),os.path.join(new_dir,"valid"))

        mapped_ids = {class_id: index for index, class_id in enumerate(new_indx_wrt_base)}
        logger.info(f"old to new class map successfull: {mapped_ids}")

        for filename in os.listdir(os.path.join(new_dir,"valid")):
            if filename.endswith('.txt'):
                filepath = os.path.join(new_dir,"valid",filename)

                with open(filepath,'r') as file:
                    lines = file.readlines()
                
                #update the class id
                updated = [self.update_id(line,mapped_ids) for line in lines]

                with open(filepath, 'w') as file:
                    file.writelines(updated)

        self.subsetandcopy(filtered_filelist, os.path.join(new_dir,"train"),new_dir,"images")
        self.subsetandcopy(filtered_filelist, os.path.join(new_dir,"valid"),new_dir,"labels")

        desired_classes = list(desired_classes)

        self.create_yaml(dir_path=new_dir,
                         class_name=desired_classes,
                         op_file=os.path.join(new_dir,"data.yaml"))

        return new_dir
    






        #train_ratio = 0.8


In [36]:
from scripts.MLOPs.exception import AppException
import sys




obj = ConfigurationManager()
config = obj.get_Dataset_config()
print(config)
if len(config.classes) != 0:
    try:
        x = DataSetPreparer(yamlconfig=config)
        x.create_dataset()
    except Exception as e:
        raise AppException(e, sys)
else:
    pass

[2024-08-01 12:32:14,379: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-08-01 12:32:14,386: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-08-01 12:32:14,390: INFO: common: created directory at artifacts]
DataSetConfig(base_Data_path='artifacts/data_ingestion/data', classes=BoxList([]), new_data_path='artifacts/data_ingestion', dataset_name='version1')
